# **Formatação e limpeza de dados - Produção Agrícola Municipal**

## **Importa bibliotecas**

In [ ]:
import numpy as np
import pandas as pd

## **Importa dados como um DataFrame do Pandas**

In [ ]:
raw = pd.read_csv("./data/RAW_PAM_1990-2019.zip", encoding="UTF-8", sep=";", decimal=",")
raw.head()

## **1- Cria tabela auxiliar, renomeia as colunas e conserta as colunas `Variável` e `Valor Variável`**

In [ ]:
columns = {
    "Cód.": "municipio_codigo",
    "Ano": "ano",
    "Área plantada ou destinada à colheita (Hectares)": "area_plantada",
    "Área colhida (Hectares)": "area_colhida",
    "Valor da produção (Mil Reais)": "valor_reais",
    "Valor da produção (Mil Cruzeiros)": "valor_cruzeiros",
    "Valor da produção (Mil Cruzeiros Reais)": "valor_cruzeiros_reais"
}

df = pd.DataFrame(columns=list(columns.keys()))

In [ ]:
mun_ids = raw["Cód."].unique()
mun_ids_total = len(mun_ids)
counter = 0
print("Total de municípios >>", len(mun_ids))
for municipio in mun_ids:
    counter += 1
    print(f"\r{counter}/{mun_ids_total}", end="")
    for ano in raw["Ano"].unique():
        temp = raw.loc[(raw["Cód."] == municipio) & (raw["Ano"] == ano)]
        temp_row = temp.iloc[0]
        dic = {
            "Cód.": municipio,
            "Ano": ano,
            "Área plantada ou destinada à colheita (Hectares)": "",
            "Área colhida (Hectares)": "",
            "Valor da produção (Mil Reais)": "",
            "Valor da produção (Mil Cruzeiros)": "",
            "Valor da produção (Mil Cruzeiros Reais)": ""
        }
        for _, row in temp.iterrows():
            dic[str(row["Variável"])] = row["Valor Variável"]
        df = df.append(dic, ignore_index=True)
df = df.rename(columns)
df = df.sort_values(by=["ano", "codigo_municipio"]).reset_index()
df["area_plantada"] = df["area_plantada"].replace(["...", "-"], np.NaN)
df["area_colhida"] = df["area_colhida"].replace(["...", "-"], np.NaN)
df["valor_reais"] = df["valor_reais"].replace(["...", "-"], np.NaN)
df["valor_cruzeiros"] = df["valor_cruzeiros"].replace(["...", "-"], np.NaN)
df["valor_cruzeiros_reais"] = df["valor_cruzeiros_reais"].replace(["...", "-"], np.NaN)
df["razao_colheita_plantio"] = df["area_colhida"]/df["area_plantada"]

## **2- Se removem os dados agregados do país (`municipio_codigo == 1`)**

In [ ]:
df = df.loc[df["municipio_codigo"] != 1]
df.head()

In [ ]:
df = df[list(columns.values())].rename(columns=columns)
df = df.sort_values(by=["municipio_codigo", "ano"]).reset_index(drop=True)
df["area_plantada"] = df["area_plantada"].replace(["...", "-"], np.NaN)
df["area_colhida"] = df["area_colhida"].replace(["...", "-"], np.NaN)
df["valor_reais"] = df["valor_reais"].replace(["...", "-"], np.NaN)
df["valor_cruzeiros"] = df["valor_cruzeiros"].replace(["...", "-"], np.NaN)
df["valor_cruzeiros_reais"] = df["valor_cruzeiros_reais"].replace(["...", "-"], np.NaN)
df["razao_plantio_colheita"] = df["area_colhida"].astype(float)/df["area_plantada"].astype(float)

In [ ]:
df.head()

In [ ]:
df.to_csv("./data/PAM.zip", sep=";", index=False, compression=dict(method="zip", archive_name="PAM.csv"))